In [2]:
!pip install faiss-cpu groq python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 11.4 MB/s eta 0:00:00


In [3]:
import pandas as pd  ### data analysis
import faiss  ### vector database
import numpy as np  ### data analysis
from groq import Groq
from sentence_transformers import SentenceTransformer  ### Word embeddings
from dotenv import load_dotenv
import os

In [4]:
GROQ_API_KEY = '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'
client = Groq(api_key=GROQ_API_KEY)

In [5]:
url = 'https://raw.githubusercontent.com/DataScience75/Top_mentor_projects_Datasets/refs/heads/main/customer_reviews.csv'
df = pd.read_csv(url)
df.head(5)

,review_id,product_id,review_text,rating
0,1,P101,Excellent battery life and superb camera quali...,5
1,2,P101,"The phone lasts two days on a single charge, g...",5
2,3,P101,Battery is good but the screen is average.,4
3,4,P101,Amazing camera but the design feels outdated.,4
4,5,P101,Very reliable battery performance.,5


In [23]:
#### You can cleaning data ###### Task
### Create embeddings ####  - ### Chunk the data in batches
### Then we can create embeddings
def chunk_text(text, chunk_size = 10):
  words = text.split()
  #print(words)
  return [" ".join(words[i:i+chunk_size])for i in range(0, len(words),chunk_size)]

In [24]:
chunks = []
metadata = []
for idx, row in df.iterrows():
  for chunk in chunk_text(row['review_text']):
    chunks.append(chunk)
    metadata.append({'product_id':row['product_id'], 'review_id':row['review_id']})

In [27]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks, convert_to_numpy = True)
embeddings

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

array([[-0.12552299,  0.04390914, -0.04582999, ...,  0.01603818,
        -0.07287002,  0.06687099],
       [ 0.04330345, -0.00582582, -0.00239525, ...,  0.03256202,
         0.01379365,  0.011328  ],
       [-0.03790098,  0.04702245,  0.0527302 , ...,  0.03159023,
        -0.02566356,  0.05701391],
       ...,
       [ 0.0348939 , -0.04569967,  0.01673002, ...,  0.0203181 ,
        -0.12329307,  0.07027455],
       [ 0.04399862, -0.01214792,  0.09306976, ..., -0.03072306,
        -0.05783972,  0.02898448],
       [-0.01532624,  0.03267721,  0.04441223, ..., -0.06343678,
        -0.09644357,  0.0835418 ]], dtype=float32)

In [28]:
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

In [32]:
### RAG  - Knowledge Base (embeddings + product_id, review_id)
### Create the LLM
### Prepare the context properly using metadata and releavnt information
### Pass that to the LLM for precise results
### Recommendation Engine  - Goal

def recommend_product(user_query,top_k = 3):
  query_vec = model.encode([user_query],convert_to_numpy=True)
  distance, indices = index.search(query_vec,top_k)

  ### Retrive Relevant reviews based on Index
  retrieved_chunks = [chunks[i] for i in indices[0]]
  retrieved_metadata = [metadata[i] for i in indices[0]]

  context = "\n".join([f"Product{m['product_id']}:{txt}" for txt, m in zip(retrieved_chunks,retrieved_metadata)])
  print(context)

  completion = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
      {"role": "system","content":"You are a helpful product recommendation assistant"},
      { "role": "user","content":f"Based on the following customer reviews :\n{context}\n\n Suggest the best product for {user_query}"}
    ],
    temperature= 0.2,
    max_tokens =200
)

  return completion.choices[0].message.content, retrieved_metadata


In [33]:
recommedations = recommend_product("I need a light weight camera phone")
print("Find the recommendations - ", recommedations)

ProductP105:Budget-friendly but camera is basic.
ProductP101:Amazing camera but the design feels outdated.
ProductP105:Low cost and works well for calls.
Find the recommendations -  ('Based on the customer reviews, it seems that ProductP101 has an "Amazing camera" which suggests it has a good camera quality. However, the design is considered "outdated". \n\nSince you need a lightweight camera phone, I would recommend ProductP105. Although the camera is considered "basic", the customer reviews also mention that it is "budget-friendly" and "works well for calls", which suggests it is lightweight and easy to use. \n\nSo, my recommendation would be ProductP105, as it seems to balance the need for a lightweight phone with a basic camera, while also being budget-friendly.', [{'product_id': 'P105', 'review_id': 22}, {'product_id': 'P101', 'review_id': 4}, {'product_id': 'P105', 'review_id': 25}])
